In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table


## set up pyspark session

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/22 15:13:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## set up config

In [3]:
# set up config
snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
dates_str_lst

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01',
 '2024-07-01',
 '2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01']

## Build Bronze Table

In [8]:
# create bronze datalake
bronze_lms_directory = "datamart/bronze/lms/"
bronze_users_directory = "datamart/bronze/users/"

if not os.path.exists(bronze_lms_directory):
    os.makedirs(bronze_lms_directory)

if not os.path.exists(bronze_users_directory):
    os.makedirs(bronze_users_directory)

In [6]:
# run bronze backfill
for date_str in dates_str_lst:
    utils.data_processing_bronze_table.process_bronze_loans(date_str, bronze_lms_directory, spark)
    utils.data_processing_bronze_table.process_bronze_users(date_str, bronze_users_directory, spark)

2023-01-01 row count: 530
saved to: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv
2023-01-01 row count (clickstream): 8974
saved to: datamart/bronze/users/bronze_users_clickstream_2023_01_01.csv
2023-01-01 row count (attributes): 530
saved to: datamart/bronze/users/bronze_users_attributes_2023_01_01.csv
2023-01-01 row count (financials): 530
saved to: datamart/bronze/users/bronze_users_financials_2023_01_01.csv
2023-02-01 row count: 1031
saved to: datamart/bronze/lms/bronze_loan_daily_2023_02_01.csv
2023-02-01 row count (clickstream): 8974
saved to: datamart/bronze/users/bronze_users_clickstream_2023_02_01.csv
2023-02-01 row count (attributes): 501
saved to: datamart/bronze/users/bronze_users_attributes_2023_02_01.csv
2023-02-01 row count (financials): 501
saved to: datamart/bronze/users/bronze_users_financials_2023_02_01.csv
2023-03-01 row count: 1537
saved to: datamart/bronze/lms/bronze_loan_daily_2023_03_01.csv
2023-03-01 row count (clickstream): 8974
saved to: datamart/bronz

In [7]:
# inspect output
loans_df = utils.data_processing_bronze_table.process_bronze_loans(date_str, bronze_lms_directory, spark)
click_df, attr_df, fin_df = utils.data_processing_bronze_table.process_bronze_users(date_str, bronze_users_directory, spark)

2024-12-01 row count: 5531
saved to: datamart/bronze/lms/bronze_loan_daily_2024_12_01.csv
2024-12-01 row count (clickstream): 8974
saved to: datamart/bronze/users/bronze_users_clickstream_2024_12_01.csv
2024-12-01 row count (attributes): 515
saved to: datamart/bronze/users/bronze_users_attributes_2024_12_01.csv
2024-12-01 row count (financials): 515
saved to: datamart/bronze/users/bronze_users_financials_2024_12_01.csv


In [8]:
loans_df.limit(5).toPandas()

,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date
0,CUS_0x100b_2024_03_01,CUS_0x100b,2024-03-01,10,9,10000,1000.0,1000.0,0.0,1000.0,2024-12-01
1,CUS_0x102e_2024_04_01,CUS_0x102e,2024-04-01,10,8,10000,1000.0,0.0,6000.0,8000.0,2024-12-01
2,CUS_0x1038_2024_10_01,CUS_0x1038,2024-10-01,10,2,10000,1000.0,1000.0,0.0,8000.0,2024-12-01
3,CUS_0x103e_2024_12_01,CUS_0x103e,2024-12-01,10,0,10000,0.0,0.0,0.0,10000.0,2024-12-01
4,CUS_0x1048_2024_02_01,CUS_0x1048,2024-02-01,10,10,10000,1000.0,0.0,9000.0,9000.0,2024-12-01


In [9]:
loans_df.dtypes

[('loan_id', 'string'),
 ('Customer_ID', 'string'),
 ('loan_start_date', 'date'),
 ('tenure', 'int'),
 ('installment_num', 'int'),
 ('loan_amt', 'int'),
 ('due_amt', 'double'),
 ('paid_amt', 'double'),
 ('overdue_amt', 'double'),
 ('balance', 'double'),
 ('snapshot_date', 'date')]

In [10]:
click_df.limit(5).toPandas()

,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date
0,145,189,109,134,196,-37,101,82,111,24,...,65,249,200,185,-83,-18,-76,30,CUS_0x1037,2024-12-01
1,40,184,187,75,192,146,38,109,353,141,...,-14,193,125,117,215,91,33,255,CUS_0x1069,2024-12-01
2,98,121,180,200,95,48,59,194,76,84,...,167,101,92,185,98,68,-60,116,CUS_0x114a,2024-12-01
3,85,96,19,47,30,39,-32,210,-81,206,...,143,94,139,237,78,187,77,33,CUS_0x1184,2024-12-01
4,98,45,155,56,112,47,52,138,153,225,...,-43,142,121,10,189,110,264,241,CUS_0x1297,2024-12-01


In [11]:
click_df.dtypes

[('fe_1', 'int'),
 ('fe_2', 'int'),
 ('fe_3', 'int'),
 ('fe_4', 'int'),
 ('fe_5', 'int'),
 ('fe_6', 'int'),
 ('fe_7', 'int'),
 ('fe_8', 'int'),
 ('fe_9', 'int'),
 ('fe_10', 'int'),
 ('fe_11', 'int'),
 ('fe_12', 'int'),
 ('fe_13', 'int'),
 ('fe_14', 'int'),
 ('fe_15', 'int'),
 ('fe_16', 'int'),
 ('fe_17', 'int'),
 ('fe_18', 'int'),
 ('fe_19', 'int'),
 ('fe_20', 'int'),
 ('Customer_ID', 'string'),
 ('snapshot_date', 'date')]

In [12]:
attr_df.limit(5).toPandas()

,Customer_ID,Name,Age,SSN,Occupation,snapshot_date
0,CUS_0x103e,Tim Kellyf,40,155-72-8070,Scientist,2024-12-01
1,CUS_0x1195,Alexk,31,822-48-3629,Manager,2024-12-01
2,CUS_0x1197,Nayako,28,799-23-8283,_______,2024-12-01
3,CUS_0x11e2,Valetkevitchr,34,809-04-1419,Musician,2024-12-01
4,CUS_0x11ec,William Schombergh,34,417-74-2163,Journalist,2024-12-01


In [13]:
fin_df.limit(5).toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date
0,CUS_0x103e,98690.8,8262.233333,4,6,9,1_,Student Loan,6,17,...,Good,706.96,26.860663,26 Years and 11 Months,No,55.004408,913.4813186573292,Low_spent_Small_value_payments,147.7376071067124,2024-12-01
1,CUS_0x1195,30429.91,2808.825833,4,6,16,2,"Auto Loan, and Auto Loan",22,17,...,Standard,362.48,33.349050,28 Years and 11 Months,No,29.914076,82.87878577514347,Low_spent_Large_value_payments,438.08972109416084,2024-12-01
2,CUS_0x1197,92300.01,7437.667500,2,4,11,3,"Credit-Builder Loan, Not Specified, and Credit...",27,9,...,_,755.17_,26.989787,18 Years and 11 Months,Yes,49236.000000,220.8621525417414,Low_spent_Large_value_payments,581.1567885447394,2024-12-01
3,CUS_0x11e2,44986.55,3689.879167,6,5,11,1,Credit-Builder Loan,0,4,...,Good,753.21,25.586286,20 Years and 0 Months,No,23.267135,43.20363344633164,High_spent_Large_value_payments,542.5171477430948,2024-12-01
4,CUS_0x11ec,14867.69,1005.974167,9,9,18,6,"Debt Consolidation Loan, Student Loan, Persona...",39,15,...,Standard,2344.06,24.344388,17 Years and 2 Months,Yes,55.459604,100.14574834721886,Low_spent_Medium_value_payments,224.99206407779144,2024-12-01


In [14]:
fin_df.dtypes

[('Customer_ID', 'string'),
 ('Annual_Income', 'string'),
 ('Monthly_Inhand_Salary', 'double'),
 ('Num_Bank_Accounts', 'int'),
 ('Num_Credit_Card', 'int'),
 ('Interest_Rate', 'int'),
 ('Num_of_Loan', 'string'),
 ('Type_of_Loan', 'string'),
 ('Delay_from_due_date', 'int'),
 ('Num_of_Delayed_Payment', 'string'),
 ('Changed_Credit_Limit', 'string'),
 ('Num_Credit_Inquiries', 'double'),
 ('Credit_Mix', 'string'),
 ('Outstanding_Debt', 'string'),
 ('Credit_Utilization_Ratio', 'double'),
 ('Credit_History_Age', 'string'),
 ('Payment_of_Min_Amount', 'string'),
 ('Total_EMI_per_month', 'double'),
 ('Amount_invested_monthly', 'string'),
 ('Payment_Behaviour', 'string'),
 ('Monthly_Balance', 'string'),
 ('snapshot_date', 'date')]

## Build Silver Table

In [5]:
import utils.data_processing_silver_table

In [9]:
# create bronze datalake
silver_loan_daily_directory = "datamart/silver/loan_daily/"
silver_users_directory = "datamart/silver/users/"

if not os.path.exists(silver_loan_daily_directory):
    os.makedirs(silver_loan_daily_directory)
    
if not os.path.exists(silver_users_directory):
    os.makedirs(silver_users_directory)

In [10]:
# run silver backfill
for date_str in dates_str_lst:
    utils.data_processing_silver_table.process_silver_loans(date_str, bronze_lms_directory, silver_loan_daily_directory, spark)
    utils.data_processing_silver_table.process_silver_users(date_str, bronze_users_directory, silver_users_directory, spark)


loaded from: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv row count: 530


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet
loaded from: datamart/bronze/users/bronze_users_clickstream_2023_01_01.csv row count: 8974


saved to: datamart/silver/users/silver_users_clickstream_2023_01_01.parquet
loaded from: datamart/bronze/users/bronze_users_financials_2023_01_01.csv row count: 530
saved to: datamart/silver/users/silver_users_financials_2023_01_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_02_01.csv row count: 1031


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_02_01.parquet
loaded from: datamart/bronze/users/bronze_users_clickstream_2023_02_01.csv row count: 8974


saved to: datamart/silver/users/silver_users_clickstream_2023_02_01.parquet
loaded from: datamart/bronze/users/bronze_users_financials_2023_02_01.csv row count: 501
saved to: datamart/silver/users/silver_users_financials_2023_02_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_03_01.csv row count: 1537
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_03_01.parquet
loaded from: datamart/bronze/users/bronze_users_clickstream_2023_03_01.csv row count: 8974


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
utils.data_processing_silver_table.process_silver_loans(date_str, bronze_lms_directory, silver_loan_daily_directory, spark).toPandas()
click_df, fin_df = utils.data_processing_silver_table.process_silver_users(date_str, bronze_users_directory, silver_users_directory, spark)

loaded from: datamart/bronze/lms/bronze_loan_daily_2023_03_01.csv row count: 1537


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_03_01.parquet
loaded from: datamart/bronze/users/bronze_users_clickstream_2023_03_01.csv row count: 8974
saved to: datamart/silver/users/silver_users_clickstream_2023_03_01.parquet
loaded from: datamart/bronze/users/bronze_users_financials_2023_03_01.csv row count: 506
saved to: datamart/silver/users/silver_users_financials_2023_03_01.parquet


In [12]:
click_df.limit(5).toPandas()

,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date
0,232,203,-71,86,243,12,79,122,103,178,...,201,37,96,63,98,69,257,3,CUS_0x1037,2023-03-01
1,-31,2,19,-52,176,-24,158,138,186,53,...,196,192,204,144,-105,4,169,98,CUS_0x1069,2023-03-01
2,139,142,127,167,29,-42,143,151,60,97,...,-132,56,150,-108,115,109,202,138,CUS_0x114a,2023-03-01
3,6,185,180,13,-60,136,40,327,171,-104,...,119,280,209,63,175,271,-82,265,CUS_0x1184,2023-03-01
4,137,208,-11,172,167,91,-5,212,290,1,...,97,220,153,-86,14,116,199,128,CUS_0x1297,2023-03-01


In [28]:
click_df.dtypes

[('fe_1', 'int'),
 ('fe_2', 'int'),
 ('fe_3', 'int'),
 ('fe_4', 'int'),
 ('fe_5', 'int'),
 ('fe_6', 'int'),
 ('fe_7', 'int'),
 ('fe_8', 'int'),
 ('fe_9', 'int'),
 ('fe_10', 'int'),
 ('fe_11', 'int'),
 ('fe_12', 'int'),
 ('fe_13', 'int'),
 ('fe_14', 'int'),
 ('fe_15', 'int'),
 ('fe_16', 'int'),
 ('fe_17', 'int'),
 ('fe_18', 'int'),
 ('fe_19', 'int'),
 ('fe_20', 'int'),
 ('Customer_ID', 'string'),
 ('snapshot_date', 'date')]

In [13]:
fin_df.limit(5).toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date
0,CUS_0x10bb,60830.921875,5044.243164,0,1,11.0,4.0,"Payday Loan, Home Equity Loan, Auto Loan, and ...",14,3,...,_,133.630005,38.712299,286,No,159.799347,404.045807,Low_spent_Small_value_payments,230.579163,2023-03-01
1,CUS_0x10eb,28315.949219,2415.662598,4,7,10.0,NaN,"Student Loan, and Payday Loan",8,8,...,Good,677.400024,26.912323,245,No,30.230997,133.048828,Low_spent_Small_value_payments,368.286407,2023-03-01
2,CUS_0x1132,46268.398438,3983.699951,5,4,20.0,NaN,"Student Loan, Debt Consolidation Loan, Student...",27,17,...,_,1447.569946,30.457222,168,Yes,103.778053,230.949158,High_spent_Small_value_payments,323.642792,2023-03-01
3,CUS_0x1135,83358.296875,6896.524902,5,3,5.0,3.0,"Auto Loan, Not Specified, and Auto Loan",6,6,...,_,727.880005,24.204260,370,No,169.936005,243.036530,High_spent_Medium_value_payments,526.679993,2023-03-01
4,CUS_0x11c7,36817.941406,3198.161621,5,4,5.0,3.0,"Home Equity Loan, Auto Loan, and Mortgage Loan",9,6,...,Good,1019.460022,35.270664,206,No,86.809921,206.514328,Low_spent_Small_value_payments,316.491913,2023-03-01


In [14]:
fin_df.dtypes

[('Customer_ID', 'string'),
 ('Annual_Income', 'float'),
 ('Monthly_Inhand_Salary', 'float'),
 ('Num_Bank_Accounts', 'int'),
 ('Num_Credit_Card', 'int'),
 ('Interest_Rate', 'float'),
 ('Num_of_Loan', 'int'),
 ('Type_of_Loan', 'string'),
 ('Delay_from_due_date', 'int'),
 ('Num_of_Delayed_Payment', 'int'),
 ('Changed_Credit_Limit', 'float'),
 ('Num_Credit_Inquiries', 'int'),
 ('Credit_Mix', 'string'),
 ('Outstanding_Debt', 'float'),
 ('Credit_Utilization_Ratio', 'float'),
 ('Credit_History_Age', 'int'),
 ('Payment_of_Min_Amount', 'string'),
 ('Total_EMI_per_month', 'float'),
 ('Amount_invested_monthly', 'float'),
 ('Payment_Behaviour', 'string'),
 ('Monthly_Balance', 'float'),
 ('snapshot_date', 'date')]

## EDA on credit labels

In [ ]:
# set dpd label definition
dpd = 30

# Path to the folder containing CSV files
folder_path = silver_loan_daily_directory

# Read all CSV files into a single DataFrame
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)

# filter only completed loans
df = df.filter(col("loan_start_date") < datetime.strptime("2024-01-01", "%Y-%m-%d"))

# create dpd flag if more than dpd
df = df.withColumn("dpd_flag", F.when(col("dpd") >= dpd, 1).otherwise(0))

# actual bads 
actual_bads_df = df.filter(col("installment_num") == 10)

# prepare for analysis
# df = df.filter(col("installment_num") < 10)

# visualise bad rate
pdf = df.toPandas()

# Group by col_A and count occurrences in col_B
grouped = pdf.groupby('mob')['dpd_flag'].mean()

# Sort the index (x-axis) of the grouped DataFrame
grouped = grouped.sort_index()

# Plotting
grouped.plot(kind='line', marker='o')

plt.title('DPD: '+ str(dpd))
plt.xlabel('mob')
plt.ylabel('bad rate')
plt.grid(True)
plt.show()


In [ ]:
df.show()

## Build gold table for labels

In [ ]:
# create bronze datalake
gold_label_store_directory = "datamart/gold/label_store/"
gold_feature_store_directory = "datamart/gold/feature_store/"

if not os.path.exists(gold_label_store_directory):
    os.makedirs(gold_label_store_directory)

In [ ]:
# run gold backfill
for date_str in dates_str_lst:
    utils.data_processing_gold_table.process_labels_gold_table(date_str, silver_loan_daily_directory, gold_label_store_directory, spark, dpd = 30, mob = 6)


In [ ]:
utils.data_processing_gold_table.process_labels_gold_table(date_str, silver_loan_daily_directory, gold_label_store_directory, spark, dpd = 30, mob = 6).dtypes


## inspect label store

In [ ]:
folder_path = gold_label_store_directory
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

In [ ]:
df.printSchema()